In [1]:
from utils.imports import *

Using TensorFlow backend.


In [2]:
def get_dirfiles(dir):
    file_list = []
    subset_path = os.listdir(dir)
    for _ in range(len(subset_path)):
        if subset_path[_] != '.DS_Store':
            file_list.append(dir + subset_path[_])
    return file_list

def train_generator(output_true,output_false):
    file_list_true = get_dirfiles(output_true)[0:9000]
    file_list_false = get_dirfiles(output_false)[0:9000]
    
    file_list_true = shuffle(file_list_true)
    file_list_false = shuffle(file_list_false)
    
    nb_true = len(file_list_true) + len(file_list_false)
    sample = np.zeros([nb_true,6,20,20])
    labels = np.zeros([nb_true,2])
    for i in tqdm(range(len(file_list_true))):       
        cc= np.load(file_list_true[i])
        sample[i] = cc
        labels[i] = [0.,1.]
    for j in tqdm(range(len(file_list_false))):
        bb= np.load(file_list_false[j])
        sample[j+len(file_list_true)] = bb 
        labels[j+len(file_list_true)] = [1.,0.]
    sample = np.expand_dims(sample, axis=1)        
    return sample,labels

def valid_generator(output_true,output_false):
    file_list_true = get_dirfiles(output_true)[-1000:]
    file_list_false = get_dirfiles(output_false)[-1000:]
    
    file_list_true = shuffle(file_list_true)
    file_list_false = shuffle(file_list_false)

    nb_true = len(file_list_true) + len(file_list_false)
    sample = np.zeros([nb_true,6,20,20])
    labels = np.zeros([nb_true,2])
  
    for i in tqdm(range(len(file_list_true))):       
        cc= np.load(file_list_true[i])
        sample[i] = cc
        labels[i] = [0.,1.]
    for j in tqdm(range(len(file_list_false))):
        bb= np.load(file_list_false[j])
        sample[j+len(file_list_true)] = bb 
        labels[j+len(file_list_true)] = [1.,0.]
    sample = np.expand_dims(sample, axis=1)        
    return sample,labels

def model_20_2():    

    inputs = Input(shape=(1, 6, 20, 20))    
    conv1 = Convolution3D(64, 3, 5, 5, activation = 'elu', border_mode='same')(inputs)
    conv1 = BatchNormalization(axis = 1)(conv1)
    conv1 = Dropout(p=0.2)(conv1)    
    conv1 = Convolution3D(64, 1, 1, 1, activation = 'elu', border_mode='same')(conv1)
    conv1 = BatchNormalization(axis = 1)(conv1)
    conv1 = Dropout(p=0.2)(conv1)   
    conv1 = Convolution3D(64, 3, 5, 5, activation = 'elu', border_mode='same')(inputs)
    conv1 = BatchNormalization(axis = 1)(conv1)
    conv1 = Dropout(p=0.2)(conv1)   
    conv1 = Convolution3D(64, 1, 5, 5, activation = 'elu', border_mode='same')(conv1)
    conv1 = BatchNormalization(axis = 1)(conv1)
    conv1 = Dropout(p=0.2)(conv1)   
    
    output = Flatten(name='flatten')(conv1)
    output = Dense(150)(output)
    output = PReLU()(output)
    output = Dropout(p=0.2)(output)   
    output = BatchNormalization()(output)
    output = Dense(2, activation='softmax', name = 'predictions')(output)
    model3d = Model(inputs, output)
    model3d.compile(loss='categorical_crossentropy', optimizer = SGD(lr=0.3, momentum=0.9), metrics = ['accuracy'])
    return model3d

def model_20():    

    inputs = Input(shape=(1, 6, 20, 20))    
    conv1 = Convolution3D(64, 3, 5, 5, activation = 'elu', border_mode='same')(inputs)
    conv1 = BatchNormalization(axis = 1)(conv1)
    conv1 = Convolution3D(64, 1, 1, 1, activation = 'elu', border_mode='same')(conv1)
    conv1 = BatchNormalization(axis = 1)(conv1)
    conv1 = Convolution3D(64, 3, 5, 5, activation = 'elu', border_mode='same')(inputs)
    conv1 = BatchNormalization(axis = 1)(conv1)
    conv1 = Convolution3D(64, 1, 5, 5, activation = 'elu', border_mode='same')(conv1)
    conv1 = BatchNormalization(axis = 1)(conv1)    
    
    output = Flatten(name='flatten')(conv1)
    output = Dense(150)(output)
    output = PReLU()(output)
    output = BatchNormalization()(output)
    output = Dense(2, activation='softmax', name = 'predictions')(output)
    model3d = Model(inputs, output)
    model3d.compile(loss='categorical_crossentropy', optimizer = Adam(lr=1e-5), metrics = ['accuracy'])
    return model3d

def fenlei_fit(name, load_check = False,batch_size=2, epochs=20,check_name = None):

    t = time.time()
    callbacks = [EarlyStopping(monitor='val_loss', patience = 200, verbose = 1),
                 ModelCheckpoint((model_paths + '{}.h5').format(name),
                                 monitor='val_loss',
                                 verbose = 0,
                                 save_best_only = True)]
    if load_check:
        check_model = (model_paths + '{}.h5').format(check_name)
        model = load_model(check_model)
    else:
        model = model_20_2()
    x,y = train_generator(output_true,output_false)
    model.fit(x=x, y=y, batch_size=batch_size, epochs=epochs,
              validation_data=valid_generator(output_true,output_false),verbose=1, callbacks=callbacks, shuffle=True) 
    return model

In [3]:
output_path = PATH['cls_train_20']
output_true = PATH['cls_train_20_true']
output_false = PATH['cls_train_20_false']
model_paths = PATH['model_paths']
model_final = PATH['model_final']

In [4]:
fenlei_fit('Fenge_6_20_20_0618', load_check = True, batch_size=200, epochs=30, check_name = 'Fenge_6_20_20_0618')

100%|██████████| 1000/1000 [00:00<00:00, 5127.39it/s]


Train on 18000 samples, validate on 2000 samples
Epoch 1/200
18000/18000 [==============================] - 89s - loss: 0.2169 - acc: 0.9130 - val_loss: 0.2141 - val_acc: 0.9210
Epoch 2/200
18000/18000 [==============================] - 87s - loss: 0.2206 - acc: 0.9148 - val_loss: 0.2445 - val_acc: 0.9055
Epoch 3/200
18000/18000 [==============================] - 87s - loss: 0.2052 - acc: 0.9211 - val_loss: 0.2194 - val_acc: 0.9170
Epoch 4/200
18000/18000 [==============================] - 88s - loss: 0.2061 - acc: 0.9210 - val_loss: 0.1968 - val_acc: 0.9305
Epoch 5/200
18000/18000 [==============================] - 87s - loss: 0.1939 - acc: 0.9252 - val_loss: 0.2153 - val_acc: 0.9175
Epoch 6/200
18000/18000 [==============================] - 87s - loss: 0.1882 - acc: 0.9274 - val_loss: 0.2003 - val_acc: 0.9285
Epoch 7/200
18000/18000 [==============================] - 87s - loss: 0.1893 - acc: 0.9261 - val_loss: 0.2073 - val_acc: 0.9260
Epoch 8/200
18000/18000 [=======================

18000/18000 [==============================] - 86s - loss: 0.0263 - acc: 0.9920 - val_loss: 0.3749 - val_acc: 0.9190
Epoch 64/200
18000/18000 [==============================] - 86s - loss: 0.0313 - acc: 0.9899 - val_loss: 0.3376 - val_acc: 0.9285
Epoch 65/200
18000/18000 [==============================] - 86s - loss: 0.0289 - acc: 0.9909 - val_loss: 0.3909 - val_acc: 0.9165
Epoch 66/200
18000/18000 [==============================] - 86s - loss: 0.0330 - acc: 0.9887 - val_loss: 0.3466 - val_acc: 0.9350
Epoch 67/200
18000/18000 [==============================] - 86s - loss: 0.0285 - acc: 0.9908 - val_loss: 0.3461 - val_acc: 0.9285
Epoch 68/200
18000/18000 [==============================] - 86s - loss: 0.0254 - acc: 0.9912 - val_loss: 0.3973 - val_acc: 0.9250
Epoch 69/200
18000/18000 [==============================] - 86s - loss: 0.0285 - acc: 0.9908 - val_loss: 0.3875 - val_acc: 0.9195
Epoch 70/200
18000/18000 [==============================] - 86s - loss: 0.0275 - acc: 0.9907 - val_loss

18000/18000 [==============================] - 86s - loss: 0.0152 - acc: 0.9952 - val_loss: 0.3791 - val_acc: 0.9250
Epoch 126/200
18000/18000 [==============================] - 86s - loss: 0.0150 - acc: 0.9956 - val_loss: 0.3876 - val_acc: 0.9315
Epoch 127/200
18000/18000 [==============================] - 86s - loss: 0.0144 - acc: 0.9955 - val_loss: 0.3962 - val_acc: 0.9270
Epoch 128/200
18000/18000 [==============================] - 86s - loss: 0.0193 - acc: 0.9947 - val_loss: 0.4505 - val_acc: 0.9275
Epoch 129/200
18000/18000 [==============================] - 86s - loss: 0.0165 - acc: 0.9952 - val_loss: 0.4233 - val_acc: 0.9250
Epoch 130/200
18000/18000 [==============================] - 86s - loss: 0.0124 - acc: 0.9968 - val_loss: 0.4314 - val_acc: 0.9215
Epoch 131/200
18000/18000 [==============================] - 86s - loss: 0.0139 - acc: 0.9956 - val_loss: 0.4695 - val_acc: 0.9200
Epoch 132/200
18000/18000 [==============================] - 86s - loss: 0.0137 - acc: 0.9962 - v

18000/18000 [==============================] - 86s - loss: 0.0107 - acc: 0.9969 - val_loss: 0.4540 - val_acc: 0.9280
Epoch 188/200
18000/18000 [==============================] - 86s - loss: 0.0101 - acc: 0.9976 - val_loss: 0.4614 - val_acc: 0.9290
Epoch 189/200
18000/18000 [==============================] - 86s - loss: 0.0094 - acc: 0.9979 - val_loss: 0.4852 - val_acc: 0.9270
Epoch 190/200
18000/18000 [==============================] - 86s - loss: 0.0094 - acc: 0.9973 - val_loss: 0.4703 - val_acc: 0.9255
Epoch 191/200
18000/18000 [==============================] - 86s - loss: 0.0132 - acc: 0.9962 - val_loss: 0.4539 - val_acc: 0.9300
Epoch 192/200
18000/18000 [==============================] - 86s - loss: 0.0103 - acc: 0.9971 - val_loss: 0.4514 - val_acc: 0.9240
Epoch 193/200
18000/18000 [==============================] - 86s - loss: 0.0108 - acc: 0.9968 - val_loss: 0.4782 - val_acc: 0.9270
Epoch 194/200
18000/18000 [==============================] - 86s - loss: 0.0101 - acc: 0.9977 - v

In [5]:
file_list_true = get_dirfiles(output_true)
file_list_false = get_dirfiles(output_false)

In [6]:
#model_pred = classifier((1, 36, 36, 36), (3, 3, 3), (2, 2, 2))
model_pred = load_model(model_paths + 'Fenge_6_20_20_0618.h5')

In [7]:
cc = []
for i in file_list_false[0:200]:
    a=np.load(i)
    a=np.expand_dims(a,0)
    a=np.expand_dims(a,0)
    cc.append(model_pred.predict(a))
count = 0
for i in cc:
    if i[0][0] > 0.9:
        count += 1
print count*1.0/len(cc)

0.87


In [8]:
cc

[array([[ 0.97026348,  0.02973649]], dtype=float32),
 array([[ 0.96584773,  0.03415221]], dtype=float32),
 array([[ 0.92281783,  0.07718219]], dtype=float32),
 array([[  9.99274313e-01,   7.25683407e-04]], dtype=float32),
 array([[ 0.26466003,  0.73534   ]], dtype=float32),
 array([[ 0.99575138,  0.00424865]], dtype=float32),
 array([[ 0.89408034,  0.10591961]], dtype=float32),
 array([[ 0.98376369,  0.01623633]], dtype=float32),
 array([[ 0.98421222,  0.01578782]], dtype=float32),
 array([[ 0.01634328,  0.98365676]], dtype=float32),
 array([[ 0.99277037,  0.00722961]], dtype=float32),
 array([[ 0.95145828,  0.04854175]], dtype=float32),
 array([[ 0.15287569,  0.84712434]], dtype=float32),
 array([[ 0.99786711,  0.00213282]], dtype=float32),
 array([[ 0.96023434,  0.03976573]], dtype=float32),
 array([[ 0.99698669,  0.0030133 ]], dtype=float32),
 array([[ 0.99776328,  0.00223668]], dtype=float32),
 array([[ 0.9646855 ,  0.03531453]], dtype=float32),
 array([[ 0.97167128,  0.02832871]],

In [9]:
cc = []
for i in file_list_true[0:200]:
    a=np.load(i)
    a=np.expand_dims(a,0)
    a=np.expand_dims(a,0)
    cc.append(model_pred.predict(a))
count = 0


In [10]:
for i in cc:
    if i[0][1] > 0.9:
        count += 1
print count*1.0/len(cc)

0.755


In [11]:
cc

[array([[ 0.35381708,  0.64618289]], dtype=float32),
 array([[ 0.3929517 ,  0.60704833]], dtype=float32),
 array([[ 0.0397889 ,  0.96021116]], dtype=float32),
 array([[ 0.0441843 ,  0.95581573]], dtype=float32),
 array([[ 0.04035529,  0.95964473]], dtype=float32),
 array([[ 0.10420643,  0.89579362]], dtype=float32),
 array([[ 0.03444644,  0.96555358]], dtype=float32),
 array([[ 0.01089699,  0.98910296]], dtype=float32),
 array([[  1.18041926e-05,   9.99988198e-01]], dtype=float32),
 array([[ 0.42278352,  0.57721645]], dtype=float32),
 array([[ 0.04140922,  0.95859081]], dtype=float32),
 array([[ 0.3540034 ,  0.64599657]], dtype=float32),
 array([[  3.08432733e-04,   9.99691606e-01]], dtype=float32),
 array([[  1.57752118e-04,   9.99842167e-01]], dtype=float32),
 array([[ 0.00519567,  0.99480432]], dtype=float32),
 array([[ 0.34400567,  0.65599436]], dtype=float32),
 array([[ 0.01286149,  0.98713845]], dtype=float32),
 array([[ 0.71489102,  0.28510895]], dtype=float32),
 array([[ 0.0010